# Design an optimized supply chain network

**Author:** Devanshi Verma
<br>
**Date**: August-15-2020
<br>
**Goal**: Determine regional production at a plant such that we have the balance between high capacity and low capacity plants while mainting regional demands and low costs

**Modelling**

- Production at regional facilities: Low and High
- Production to other regions
- Productional facilities open or close

**Decision Variables**

 $$X_{i,j} = \text{Quantity produced at i and shipped to j}$$
<br>
$$\begin{Bmatrix} Y_{i,s}=1 & \text{ if plant location i and capacity s is open}  \\ Y_{i,s}=0 & \text{ if plant location i and capacity s is closed} \end{Bmatrix}\ $$


**Objective Function**
$$min \sum _{i=1}^{n} f_{is}y_{is} +\sum _{i=1}^{n}\sum _{i=1}^{m} v_{ij}x_{ij}$$
<br>
- i.e. Sum of (fixed costs * Yis) over production facilities + Sum of (variablecost X xij) over productional facilities and number of markets

**Constraints**
$$\sum _{i=1}^{n} x_{ij}  = D_{j}$$
- Where j=1 to m and D=demand
- n= production facilities
- m= markets

$$Y_{ih}+Y_{il}<=1$$
- h= high capacity
- l=low capacity

$$\sum _{i=1}^{n} x_{ij}<=\sum _{i=1}^{n}K_{is}y_{is}$$
- K= Potiential Capacity





**Type of Optimization problem**: Mixed Integer programming since X is continous whereas Y is constrained to integer values.

- More on the problem can be found at :http://web.mit.edu/15.053/www/AMP-Chapter-09.pdf

In [1]:
#importing the libraries
import pandas as pd
from pulp import *

In [2]:
#importing the datasets
demand=pd.read_csv("/Users/devanshiverma/Desktop/UChicago/02-Quarter/Optimization_final_Project/Data/Demand.csv")
fix_cost=pd.read_csv("/Users/devanshiverma/Desktop/UChicago/02-Quarter/Optimization_final_Project/Data/fix_cost.csv")
var_cost=pd.read_csv("/Users/devanshiverma/Desktop/UChicago/02-Quarter/Optimization_final_Project/Data/var_cost.csv")
cap=pd.read_csv("/Users/devanshiverma/Desktop/UChicago/02-Quarter/Optimization_final_Project/Data/cap.csv")

In [3]:
#Setting indexes
fix_cost=fix_cost.set_index("Supply_Region")
var_cost=var_cost.set_index("Supply_Region")
demand=demand.set_index("Supply_Region")
cap=cap.set_index("Supply_Region")

**Datasets**

In [4]:
fix_cost.head()

,Low_Cap,High_Cap
Supply_Region,,
U.S,6500,9500
Brazil,3230,4730
Canada,4980,7270
Mexico,1000,1460
Argentina,1200,1752


In [5]:
demand.head()

,Dmd
Supply_Region,
U.S,3653
Brazil,700
Canada,2587
Mexico,652
Argentina,1093


In [6]:
var_cost.head()

,U.S,Brazil,Canada,Mexico,Argentina,Chile,Colombia,Puerto Rico,Uruguay
Supply_Region,,,,,,,,,
U.S,0.6,7.0,2.3,1.9,4.0,3.2,4.1,3.5,4.60
Brazil,7.0,0.6,9.2,7.0,2.8,1.5,3.2,4.0,1.00
Canada,2.3,9.2,0.6,3.6,11.2,5.3,6.4,5.4,5.50
Mexico,1.9,7.0,3.6,0.7,7.4,3.7,3.7,3.7,4.00
Argentina,4.0,2.8,11.2,7.4,0.5,0.6,7.0,6.3,0.65


In [7]:
cap.head()

,Low_Cap,High_Cap
Supply_Region,,
U.S,500.0,1500
Brazil,500.0,1500
Canada,500.0,1500
Mexico,500.0,1500
Argentina,500.0,1500


**Model**

In [8]:
#Initialise the model
model=LpProblem("Capacitedplantlocation",LpMinimize)

#Decision Variables
loc=list(demand.index)
size=['Low_Cap','High_Cap']

x=LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],lowBound=0,upBound=None,cat='Continuous')
y=LpVariable.dicts("plant_", [(i,s) for i in loc for s in size],cat='Binary')

#Objective Function
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc])
        + lpSum([var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc]))

# Define the constraints
for j in loc:
    model += lpSum([x[i,j] for i in loc]) == demand.loc[j,'Dmd']
    
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s] * y[i,s] for s in size])
    
for i in loc:
    model +=   y[i,'High_Cap']+ y[i,'Low_Cap'] <= 1
    
model

Capacitedplantlocation:
MINIMIZE
1752*plant__('Argentina',_'High_Cap') + 1200*plant__('Argentina',_'Low_Cap') + 4730*plant__('Brazil',_'High_Cap') + 3230*plant__('Brazil',_'Low_Cap') + 7270*plant__('Canada',_'High_Cap') + 4980*plant__('Canada',_'Low_Cap') + 2336*plant__('Chile',_'High_Cap') + 1600*plant__('Chile',_'Low_Cap') + 1460*plant__('Colombia',_'High_Cap') + 1000*plant__('Colombia',_'Low_Cap') + 1460*plant__('Mexico',_'High_Cap') + 1000*plant__('Mexico',_'Low_Cap') + 4672*plant__('Puerto_Rico',_'High_Cap') + 3200*plant__('Puerto_Rico',_'Low_Cap') + 9500*plant__('U.S',_'High_Cap') + 6500*plant__('U.S',_'Low_Cap') + 2482*plant__('Uruguay',_'High_Cap') + 1700*plant__('Uruguay',_'Low_Cap') + 0.5*production__('Argentina',_'Argentina') + 2.8*production__('Argentina',_'Brazil') + 11.2*production__('Argentina',_'Canada') + 0.6*production__('Argentina',_'Chile') + 7.0*production__('Argentina',_'Colombia') + 7.4*production__('Argentina',_'Mexico') + 6.3*production__('Argentina',_'Puerto_R

In [9]:
#Solving the model
model.solve()

print("Status of the mode is: {}".format(LpStatus[model.status]))

Status of the mode is: Optimal


In [10]:
min_value=pulp.value(model.objective)
min_value

41377.259999999995

**Output**

In [14]:
import re
for i in model.variables():
    print(i,i.varValue)

plant__('Argentina',_'High_Cap') 1.0
plant__('Argentina',_'Low_Cap') 0.0
plant__('Brazil',_'High_Cap') 0.0
plant__('Brazil',_'Low_Cap') 0.0
plant__('Canada',_'High_Cap') 1.0
plant__('Canada',_'Low_Cap') 0.0
plant__('Chile',_'High_Cap') 1.0
plant__('Chile',_'Low_Cap') 0.0
plant__('Colombia',_'High_Cap') 1.0
plant__('Colombia',_'Low_Cap') 0.0
plant__('Mexico',_'High_Cap') 1.0
plant__('Mexico',_'Low_Cap') 0.0
plant__('Puerto_Rico',_'High_Cap') 1.0
plant__('Puerto_Rico',_'Low_Cap') 0.0
plant__('U.S',_'High_Cap') 0.0
plant__('U.S',_'Low_Cap') 0.0
plant__('Uruguay',_'High_Cap') 1.0
plant__('Uruguay',_'Low_Cap') 0.0
production__('Argentina',_'Argentina') 1093.0
production__('Argentina',_'Brazil') 0.0
production__('Argentina',_'Canada') 0.0
production__('Argentina',_'Chile') 262.0
production__('Argentina',_'Colombia') 0.0
production__('Argentina',_'Mexico') 0.0
production__('Argentina',_'Puerto_Rico') 0.0
production__('Argentina',_'U.S') 112.0
production__('Argentina',_'Uruguay') 0.0
productio

In [15]:
#converting the output into a pandas dataframe to be used for Travelling Salesman Problem
loc1=[]
loc2=[]
quan=[]
for i in loc:
    for j in loc:
        loc1.append(i)
        loc2.append(j)
        quan.append(x[i,j].varValue)

d=pd.DataFrame(loc1,columns=['Start'])
d['Destination']=loc2
d['quantity']=quan
d   

,Start,Destination,quantity
0,U.S,U.S,0.0
1,U.S,Brazil,0.0
2,U.S,Canada,0.0
3,U.S,Mexico,0.0
4,U.S,Argentina,0.0
...,...,...,...
76,Uruguay,Argentina,0.0
77,Uruguay,Chile,0.0
78,Uruguay,Colombia,0.0
79,Uruguay,Puerto Rico,0.0


In [16]:
d.head(30)

,Start,Destination,quantity
0,U.S,U.S,0.0
1,U.S,Brazil,0.0
2,U.S,Canada,0.0
3,U.S,Mexico,0.0
4,U.S,Argentina,0.0
5,U.S,Chile,0.0
6,U.S,Colombia,0.0
7,U.S,Puerto Rico,0.0
8,U.S,Uruguay,0.0
9,Brazil,U.S,0.0


In [17]:
loc1=[]
cap_p=[]
status=[]
for i in loc:
    for s in size:
        loc1.append(i)
        cap_p.append(s)
        status.append(y[i,s].varValue)
a=pd.DataFrame(loc1,columns=['location'])
a['capacity']=cap_p
a['status']=status
a 

,location,capacity,status
0,U.S,Low_Cap,0.0
1,U.S,High_Cap,0.0
2,Brazil,Low_Cap,0.0
3,Brazil,High_Cap,0.0
4,Canada,Low_Cap,0.0
5,Canada,High_Cap,1.0
6,Mexico,Low_Cap,0.0
7,Mexico,High_Cap,1.0
8,Argentina,Low_Cap,0.0
9,Argentina,High_Cap,1.0


#### Creating folium Map

In [18]:
from geopy.geocoders import Nominatim
geolocator=Nominatim(user_agent="Optimization_Project")
a['Lat']=a['location'].apply(lambda x: geolocator.geocode(x).latitude)
a['Long']=a['location'].apply(lambda x: geolocator.geocode(x).longitude)

In [20]:
import folium
from folium.plugins import HeatMap
base_map2=folium.Map(location=[19.432630, -99.133178], zoom_start=2,tiles='cartodbpositron')
for idx, row in a.iterrows():
    location=[row['Lat'],row['Long']]
    if row['status']==1:
        popup= '<strong>'+"OPEN"+"\n"+"\n"+row['capacity']+'</strong>'
        marker=folium.Marker(location,popup=popup,icon=folium.Icon(color='green'))
    else:
        popup= '<strong>'+row['location']+'</strong>'
        marker=folium.Marker(location,popup=popup,icon=folium.Icon(color='blue'))
    marker.add_to(base_map2)
base_map2

In [26]:
base_map2.save('base_map2.html')

# Cost reduction percentage

In [29]:
#Initialise the model
model=LpProblem("Capacitedplantlocation",LpMaximize)

#Decision Variables
loc=list(demand.index)
size=['Low_Cap','High_Cap']

x=LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],lowBound=0,upBound=None,cat='Continuous')
y=LpVariable.dicts("plant_", [(i,s) for i in loc for s in size],cat='Binary')

#Objective Function
model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc])
        + lpSum([var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc]))

# Define the constraints
for j in loc:
    model += lpSum([x[i,j] for i in loc]) == demand.loc[j,'Dmd']
    
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s] * y[i,s] for s in size])
    
for i in loc:
    model +=   y[i,'High_Cap']+ y[i,'Low_Cap'] <= 1
    
model

#Solving the model
model.solve()

print("Status of the mode is: {}".format(LpStatus[model.status]))

max_value=pulp.value(model.objective)

Status of the mode is: Optimal


In [31]:
print("the improvement is: {} %".format(((max_value-min_value)/max_value)*100))

the improvement is: 60.439326563529306 %
